In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/ssssssss/ss.pkl
/kaggle/input/fork-of-lgb-regression-16-12-16-13/sub_lgb_after_ann_normal_tdn12_tsn16_tdan13_tsan16.csv
/kaggle/input/fork-of-lgb-regression-16-12-16-13/model_lat_16_12_16_13.pkl
/kaggle/input/fork-of-lgb-regression-16-12-16-13/__results__.html
/kaggle/input/fork-of-lgb-regression-16-12-16-13/__notebook__.ipynb
/kaggle/input/fork-of-lgb-regression-16-12-16-13/__output__.json
/kaggle/input/fork-of-lgb-regression-16-12-16-13/model_lng_16_12_16_13.pkl
/kaggle/input/fork-of-lgb-regression-16-12-16-13/custom.css
/kaggle/input/google-smartphone-decimeter-challenge/sample_submission.csv
/kaggle/input/google-smartphone-decimeter-challenge/baseline_locations_train.csv
/kaggle/input/google-smartphone-decimeter-challenge/baseline_locations_test.csv
/kaggle/input/google-smartphone-decimeter-challenge/metadata/constellation_type_mapping.csv
/kaggle/input/google-smartphone-decimeter-challenge/metadata/accumulated_delta_range_state_bit_map.json
/kaggle/input/google-smart

In [2]:
! pip3 install simdkalman
from pathlib import Path
import random 
from tqdm import tqdm
import joblib
import tensorflow as tf
from scipy.interpolate import interp1d 
import simdkalman 
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error # 均方误差
from sklearn.metrics import mean_absolute_error # 平方绝对误差
import lightgbm as lgb 
import pandas as pd
import numpy as np

In [3]:
import keras 
import os
import haversine 
import matplotlib.pyplot as plt
from matplotlib_venn import venn2, venn2_circles
import seaborn as sns
import pathlib
import plotly
from datetime import datetime 
import plotly.express as px 
from sklearn.linear_model import Ridge, RidgeCV
import matplotlib.pyplot as plt 
from sklearn import linear_model  
from sklearn.ensemble import IsolationForest 
import warnings
from sklearn.model_selection import KFold
from numba import jit
import joblib
import numpy as np
import pandas as pd
# import modin.pandas as pd
from pathlib import Path
from tqdm import tqdm
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import lightgbm as lgb 
from lightgbm import LGBMRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error
from scipy.stats import skew
from scipy.stats import kurtosis
from scipy.stats import mode 
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler 
from tensorflow.keras.callbacks import ReduceLROnPlateau, ModelCheckpoint

from tensorflow.keras.layers import LeakyReLU, Input, concatenate, Dense, LSTM, BatchNormalization, Dropout, Concatenate, Lambda, GaussianNoise, Activation, Reshape, Embedding, Add, Subtract, LeakyReLU, Flatten, AdditiveAttention
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.losses import BinaryCrossentropy
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.layers.experimental.preprocessing import Normalization
import tensorflow as tf
from tensorflow_addons.optimizers import AdamW
from tensorflow.keras.metrics import AUC 
from tensorflow.keras.activations import swish, relu
from tensorflow.keras.backend import square 
from keras.engine.topology import Layer 
from keras import backend as K
import warnings 
from sklearn.linear_model import Lasso, LassoCV, LassoLarsCV
import random 
from sklearn.ensemble import IsolationForest 
from sklearn.preprocessing import StandardScaler
import simdkalman
from pathlib import Path
from tqdm import tqdm

from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.ensemble import RandomForestRegressor
import gc 
from sklearn.metrics import r2_score
seeds = [2022]
warnings.filterwarnings("ignore")

In [4]:
pd.set_option('expand_frame_repr', False)


def get_union(dir_name, file_name):
    print("Reading", file_name, "from", dir_name, "...")
    datas = []
    for path in tqdm(Path(dir_name).rglob(file_name)):
        datas.append(pd.read_csv(path, low_memory=False, index_col=False))
    return pd.concat(datas)


def check_random(SEED=2021):
    os.environ['TF_DETERMINISTIC_OPS'] = '1'
    os.environ['PYTHONHASHSEED'] = str(SEED)
    random.seed(SEED)
    np.random.seed(SEED)
    tf.random.set_seed(SEED)

def calc_haversine(lat1, lng1, lat2, lng2):
    """from
    https://www.kaggle.com/dehokanta/baseline-post-processing-by-outlier-correction
    """
    RADIUS = 6_367_000
    lat1, lng1, lat2, lng2 = map(np.radians, [lat1, lng1, lat2, lng2])
    dlat = lat2 - lat1
    dlng = lng2 - lng1
    a = np.sin(dlat / 2) ** 2 + \
        np.cos(lat1) * np.cos(lat2) * np.sin(dlng / 2) ** 2
    dist = 2 * RADIUS * np.arcsin(a ** 0.5)
    return dist



class KF():
    def __init__(self):
        T = 1.0
        state_transition = np.array([[1, 0, T, 0, 0.5 * T ** 2, 0],
                                     [0, 1, 0, T, 0, 0.5 * T ** 2],
                                     [0, 0, 1, 0, T, 0],
                                     [0, 0, 0, 1, 0, T],
                                     [0, 0, 0, 0, 1, 0],
                                     [0, 0, 0, 0, 0, 1]])
        process_noise = np.diag([1e-5, 1e-5, 5e-6, 5e-6, 1e-6, 1e-6]) + np.ones((6, 6)) * 1e-9
        observation_model = np.array([[1, 0, 0, 0, 0, 0], [0, 1, 0, 0, 0, 0]])
        observation_noise = np.diag([5e-5, 5e-5]) + np.ones((2, 2)) * 1e-9
        self.kf = simdkalman.KalmanFilter(
            state_transition=state_transition,
            process_noise=process_noise,
            observation_model=observation_model,
            observation_noise=observation_noise)

    def apply_kf_smoothing(self, df, suffix):
        phones = df["phone"].drop_duplicates().tolist()
        for phone in tqdm(phones):
            cond = df['phone'] == phone
            tmp = df[cond].copy()
            tmp[0] = tmp["millisSinceGpsEpoch"] // 1000
            tmp = tmp.merge(pd.DataFrame(range(tmp[0].min(), tmp[0].max() + 1)), on=[0], how="right")
            tmp_np = tmp[['latDeg', 'lngDeg']].to_numpy()
            nan_idxs = tmp[tmp["millisSinceGpsEpoch"].isnull()].index.to_list()
            tmp_np = tmp_np.reshape(1, len(tmp_np), 2)
            smoothed = self.kf.smooth(tmp_np).states.mean
            smoothed = np.delete(smoothed, list(nan_idxs), 1)
            df.loc[cond, 'latDeg' + suffix] = smoothed[0, :, 0]
            df.loc[cond, 'lngDeg' + suffix] = smoothed[0, :, 1] 

def mean_with_other_phones(df):
    """
    https://www.kaggle.com/bpetrb/adaptive-gauss-phone-mean
    """
    collections_list = df[['collectionName']].drop_duplicates().to_numpy()

    for collection in collections_list:
        phone_list = df[df['collectionName'].to_list() == collection][['phoneName']].drop_duplicates().to_numpy()

        phone_data = {}
        corrections = {}
        for phone in phone_list:
            cond = np.logical_and(df['collectionName'] == collection[0], df['phoneName'] == phone[0]).to_list()
            phone_data[phone[0]] = df[cond][['millisSinceGpsEpoch', 'latDeg', 'lngDeg']].to_numpy()

        for current in phone_data:
            correction = np.ones(phone_data[current].shape, dtype=np.float)
            correction[:, 1:] = phone_data[current][:, 1:]

            # Telephones data don't complitely match by time, so - interpolate.
            for other in phone_data:
                if other == current:
                    continue

                loc = interp1d(phone_data[other][:, 0],
                               phone_data[other][:, 1:],
                               axis=0,
                               kind='linear',
                               copy=False,
                               bounds_error=None,
                               fill_value='extrapolate',
                               assume_sorted=True)

                start_idx = 0
                stop_idx = 0
                for idx, val in enumerate(phone_data[current][:, 0]):
                    if val < phone_data[other][0, 0]:
                        start_idx = idx
                    if val < phone_data[other][-1, 0]:
                        stop_idx = idx

                if stop_idx - start_idx > 0:
                    correction[start_idx:stop_idx, 0] += 1
                    correction[start_idx:stop_idx, 1:] += loc(phone_data[current][start_idx:stop_idx, 0])

            correction[:, 1] /= correction[:, 0]
            correction[:, 2] /= correction[:, 0]

            corrections[current] = correction.copy()

        for phone in phone_list:
            cond = np.logical_and(df['collectionName'] == collection[0], df['phoneName'] == phone[0]).to_list()

            df.loc[cond, ['latDeg', 'lngDeg']] = corrections[phone[0]][:, 1:]

    return df

In [5]:
def parse_gnsslog(data_dir, output_dir):
    header_of_table_named = {
        "UncalMag":
            "utcTimeMillis,elapsedRealtimeNanos,UncalMagXMicroT,UncalMagYMicroT,UncalMagZMicroT\n",
        "UncalAccel":
            "utcTimeMillis,elapsedRealtimeNanos,UncalAccelXMps2,UncalAccelYMps2,UncalAccelZMps2\n",
        "UncalGyro":
            "utcTimeMillis,elapsedRealtimeNanos,UncalGyroXRadPerSec,UncalGyroYRadPerSec,UncalGyroZRadPerSec\n",
        "Status":
            "UnixTimeMillis,SignalCount,SignalIndex,ConstellationType,Svid,CarrierFrequencyHz,Cn0DbHz,"
            "AzimuthDegrees,ElevationDegrees,UsedInFix,HasAlmanacData,HasEphemerisData\n",
        "Raw":
            "utcTimeMillis,TimeNanos,LeapSecond,TimeUncertaintyNanos,FullBiasNanos,BiasNanos,BiasUncertaintyNanos,"
            "DriftNanosPerSecond,DriftUncertaintyNanosPerSecond,HardwareClockDiscontinuityCount,"
            "Svid,TimeOffsetNanos,State,ReceivedSvTimeNanos,ReceivedSvTimeUncertaintyNanos,Cn0DbHz,"
            "PseudorangeRateMetersPerSecond,PseudorangeRateUncertaintyMetersPerSecond,AccumulatedDeltaRangeState,"
            "AccumulatedDeltaRangeMeters,AccumulatedDeltaRangeUncertaintyMeters,CarrierFrequencyHz,CarrierCycles,"
            "CarrierPhase,CarrierPhaseUncertainty,MultipathIndicator,SnrInDb,ConstellationType,AgcDb\n",
        "Fix":
            "Provider,LatitudeDegrees,LongitudeDegrees,AltitudeMeters,SpeedMps,AccuracyMeters,BearingDegrees,"
            "UnixTimeMillis,SpeedAccuracyMps,BearingAccuracyDegrees\n"
    }
#     for part in ["train", "test"]:
    for part in ["test"]:
        print(part, "gnsslog parsing ...")
        for file_name in Path(data_dir / part).rglob("*GnssLog.txt"):
            print(file_name)
            with open(str(file_name)) as f_open:
                datalines = f_open.readlines()
            f_named = {}
            for f_name in header_of_table_named.keys():
                dir_pth = Path('/'.join(str(file_name.parent).split("/")[-3:]))
                Path(dir_pth.parent.parent).mkdir(exist_ok=True)
                Path(dir_pth.parent).mkdir(exist_ok=True)
                Path(dir_pth).mkdir(exist_ok=True)
                output_filename = dir_pth / (f_name + ".csv")
                if not (output_filename).exists():
                    f_named[f_name] = open(output_filename, "w")
            if f_named:
                collectionName = file_name.parent.parent.name
                phoneName = file_name.parent.name
                for f_name, f in f_named.items():
                    f.write("collectionName,phoneName," + header_of_table_named[f_name])
                for dataline in tqdm(datalines):
                    for f_name in f_named.keys():
                        if dataline.startswith(f_name):
                            f_named[f_name].write(collectionName)
                            f_named[f_name].write(',')
                            f_named[f_name].write(phoneName)
                            f_named[f_name].write(',')
                            f_named[f_name].write(dataline[len(f_name) + 1:])
                            break
                for f in f_named.values():
                    f.close()
    print("finish")

In [6]:
DATA_DIR = Path("../input/google-smartphone-decimeter-challenge")
OUTPUT_DIR = Path("./")

# 读取
# gt_data = get_union(DATA_DIR / "train", "ground_truth.csv")
# train_data = pd.read_csv(DATA_DIR / "baseline_locations_train.csv")
test_data = pd.read_csv(DATA_DIR / "baseline_locations_test.csv")

parse_gnsslog(DATA_DIR, OUTPUT_DIR)

# 手机类别
idx_of_phonename = {phoneName: idx for idx, phoneName in enumerate(test_data.phoneName.drop_duplicates().to_list())}
test_data["phoneCat"] = test_data["phoneName"].map(idx_of_phonename)


def data_processing(data):
    # 时间周期性特征
    data[["month", "day"]] = data['collectionName'].str.split('-', expand=True)[[1, 2]]
    data["ratio_of_year"] = (30 * (data.month.astype(int) - 1) + data.day.astype(int)) / 365

    # 相对于相邻帧平均值的差值的绝对值
    offsets = [1, 2, 3, 4, 5]
    groups = data[["phone", "latDeg", "lngDeg"]].groupby("phone")
    for offset in offsets:
        data[["mean_latDeg+-" + str(offset), "mean_lngDeg+-" + str(offset)]] = \
            groups.rolling(window=3, min_periods=2, center=True).mean().values
        data["latDeg_mean_delta_" + str(offset)] = np.abs(data["mean_latDeg+-" + str(offset)] - data["latDeg"])
        data["lngDeg_mean_delta_" + str(offset)] = np.abs(data["mean_lngDeg+-" + str(offset)] - data["lngDeg"])

    # 添加前后差值特征
    data[["latDeg-1", "lngDeg-1"]] = groups.shift(1)
    data[["latDeg+1", "lngDeg+1"]] = groups.shift(-1)
    data["latDeg_pre_increment"] = data["latDeg"] - data["latDeg-1"]
    data["lngDeg_pre_increment"] = data["lngDeg"] - data["lngDeg-1"]
    data["latDeg_post_increment"] = data["latDeg+1"] - data["latDeg"]
    data["lngDeg_post_increment"] = data["lngDeg+1"] - data["lngDeg"]
    data["dist_pre"] = calc_haversine(data["latDeg"], data["lngDeg"], data["latDeg-1"], data["lngDeg-1"])
    data["dist_post"] = calc_haversine(data["latDeg"], data["lngDeg"], data["latDeg+1"], data["lngDeg+1"])

    # 没有前一帧，用后一帧来代替
    nan_idxs = data[data.latDeg_pre_increment.isnull()].index
    data.loc[nan_idxs, ["latDeg_pre_increment", "lngDeg_pre_increment", "dist_pre"]] = \
        data.loc[nan_idxs + 1, ["latDeg_pre_increment", "lngDeg_pre_increment", "dist_pre"]].values
    nan_idxs = data[data.latDeg_post_increment.isnull()].index
    data.loc[nan_idxs, ["latDeg_post_increment", "lngDeg_post_increment", "dist_post"]] = \
        data.loc[nan_idxs - 1, ["latDeg_post_increment", "lngDeg_post_increment", "dist_post"]].values

    # 添加前后差值绝对均值特征
    data["latDeg_pre_post_mean_abs_delta"] = (np.abs(data["latDeg_pre_increment"]) +
                                              np.abs(data["latDeg_post_increment"])) / 2
    data["lngDeg_pre_post_mean_abs_delta"] = (np.abs(data["lngDeg_pre_increment"]) +
                                              np.abs(data["lngDeg_post_increment"])) / 2
    data["pre_post_mean_abs_dist"] = (np.abs(data["dist_pre"]) + np.abs(data["dist_post"])) / 2

    # 同一时间不同设备的统计数值
    col_names = ["latDeg", "lngDeg", "heightAboveWgs84EllipsoidM"]
    fn_names = ["max", "min", "mean", "sum", "count"]
    groups = data.groupby("millisSinceGpsEpoch")
    for fn_name_ in fn_names:
        data[[fn_name_ + '_' + col_name + "_with_same_millisSinceGpsEpoch" for col_name in col_names]] = \
            groups[col_names].transform(fn_name_).values

    # 与同时间所有手机位置均值的距离
    data["dist_between_baseline_and_phone_mean"] = calc_haversine(data.latDeg, data.lngDeg,
                                                                  data.mean_latDeg_with_same_millisSinceGpsEpoch,
                                                                  data.mean_lngDeg_with_same_millisSinceGpsEpoch)

data_processing(test_data)
test_derived_data = get_union(DATA_DIR / "test", "*_derived.csv")

def derived_data_processing(derived_data, data):
    # 添加correctedPrm特征
    derived_data["correctedPrm"] = derived_data["rawPrM"] + derived_data["satClkBiasM"] - \
                                   derived_data["isrbM"] - derived_data["ionoDelayM"] - \
                                   derived_data["tropoDelayM"]

    # 信号类型与卫星id
    derived_data["signalType_svid"] = derived_data["signalType"] + '_' + derived_data["svid"].astype("string")

    # 时间对齐
    data["millisSinceGpsEpoch/1000_round"] = np.round(data["millisSinceGpsEpoch"] / 1000).astype(np.int64)
    derived_data["millisSinceGpsEpoch/1000_round"] = np.round(derived_data["millisSinceGpsEpoch"] / 1000).astype(
        np.int64)

    # data和derived_data合并
    groups = derived_data.groupby(["collectionName", "phoneName", "millisSinceGpsEpoch/1000_round"])
    data = data.merge(
        groups["signalType_svid"].agg(lambda group: group.values),
        on=["collectionName", "phoneName", "millisSinceGpsEpoch/1000_round"], how="left"
    )
    data = data.merge(
        groups[["correctedPrm", "rawPrUncM", "satClkDriftMps"]].mean().rename(columns={
            "correctedPrm": "correctedPrm_avg",
            "rawPrUncM": "rawPrUncM_avg",
            "satClkDriftMps": "satClkDriftMps_avg"
        }),
        on=["collectionName", "phoneName", "millisSinceGpsEpoch/1000_round"], how="left"
    )
    data = data.merge(
        groups[["correctedPrm", "rawPrUncM", "satClkDriftMps"]].std().rename(columns={
            "correctedPrm": "correctedPrm_std",
            "rawPrUncM": "rawPrUncM_std",
            "satClkDriftMps": "satClkDriftMps_std"
        }),
        on=["collectionName", "phoneName", "millisSinceGpsEpoch/1000_round"], how="left"
    )

    return data


test_data = derived_data_processing(test_derived_data, test_data)
del test_derived_data
gc.collect()

# 分桶去null
train_data = pd.read_csv('../input/bucket-traindata/train_data.csv')
num_buckets = 20
attr_names = ["correctedPrm", "rawPrUncM", "satClkDriftMps"]
concat_data = pd.concat([train_data, test_data])
# concat_data = test_data
for fn_name in ["avg", "std"]:
    for attr_name in attr_names:
        concat_data[attr_name + '_' + fn_name + "_bucketized"] = pd.qcut(concat_data[attr_name + '_' + fn_name],
                                                                  num_buckets, labels=False)
        concat_data.loc[concat_data[attr_name + '_' + fn_name + "_bucketized"].isnull(),
                 attr_name + '_' + fn_name + "_bucketized"] = num_buckets
        concat_data[attr_name + '_' + fn_name + "_bucketized"] = \
            concat_data[attr_name + '_' + fn_name + "_bucketized"].astype(np.int64)
        test_data[attr_name + '_' + fn_name + "_bucketized"] = \
            concat_data[-len(test_data):][attr_name + '_' + fn_name + "_bucketized"]


test_raw_data = get_union(OUTPUT_DIR / "test", "Raw.csv")


def raw_data_processing(raw_data, data):
    raw_data["millisSinceGpsEpoch"] = np.round((raw_data.TimeNanos - raw_data.FullBiasNanos) / 1000000).\
        astype(np.int64)
    raw_data["millisSinceGpsEpoch/1000_round"] = np.round(raw_data["millisSinceGpsEpoch"] / 1000).astype(np.int64)
    groups = raw_data.groupby(["collectionName", "phoneName", "millisSinceGpsEpoch/1000_round"])
    data = data.merge(
        groups[["Cn0DbHz", "BiasUncertaintyNanos"]].mean().rename(columns={
            "Cn0DbHz": "Cn0DbHz_avg",
            "BiasUncertaintyNanos": "BiasUncertaintyNanos_avg",
        }),
        on=["collectionName", "phoneName", "millisSinceGpsEpoch/1000_round"], how="left"
    )
    data = data.merge(
        groups[["Cn0DbHz", "BiasUncertaintyNanos"]].std().rename(columns={
            "Cn0DbHz": "Cn0DbHz_std",
            "BiasUncertaintyNanos": "BiasUncertaintyNanos_std",
        }),
        on=["collectionName", "phoneName", "millisSinceGpsEpoch/1000_round"], how="left"
    )
    data = data.merge(
        groups["utcTimeMillis"].first(),
        on=["collectionName", "phoneName", "millisSinceGpsEpoch/1000_round"], how="left"
    )
    return data


test_data = raw_data_processing(test_raw_data, test_data)
del test_raw_data
gc.collect()

test_gyro_data = get_union(OUTPUT_DIR / "test", "UncalGyro.csv")
test_mag_data = get_union(OUTPUT_DIR / "test", "UncalMag.csv")
test_accel_data = get_union(OUTPUT_DIR / "test", "UncalAccel.csv")

def sensor_data_processing(gyro_data, mag_data, accel_data, data):
    data["utcTimeMillis/1000_round"] = np.round(data.utcTimeMillis / 1000).astype(np.int64)
    axises = ['X', 'Y', 'Z']
    prefix_suffix_df_dicts = [
        {"prefix": "UncalAccel", "suffix": "Mps2", "sensor_data": accel_data},
        {"prefix": "UncalGyro", "suffix": "RadPerSec", "sensor_data": gyro_data},
        {"prefix": "UncalMag", "suffix": "MicroT", "sensor_data": mag_data}
    ]
    for prefix_suffix_df_dict in prefix_suffix_df_dicts:
        prefix = prefix_suffix_df_dict["prefix"]
        suffix = prefix_suffix_df_dict["suffix"]
        sensor_data = prefix_suffix_df_dict["sensor_data"]
        sensor_data["utcTimeMillis"] = sensor_data.utcTimeMillis.astype(np.int64)
        sensor_data["utcTimeMillis/1000_round"] = np.round(sensor_data.utcTimeMillis / 1000).astype(np.int64)
        groups = sensor_data.groupby(["collectionName", "phoneName", "utcTimeMillis/1000_round"])
        data = data.merge(groups[[prefix + axis + suffix for axis in axises]].mean().rename(columns={
            prefix + axis + suffix: prefix + axis + suffix + "_avg" for axis in axises
        }), on=["collectionName", "phoneName", "utcTimeMillis/1000_round"], how="left")
        data = data.merge(groups[[prefix + axis + suffix for axis in axises]].std().rename(columns={
            prefix + axis + suffix: prefix + axis + suffix + "_std" for axis in axises
        }), on=["collectionName", "phoneName", "utcTimeMillis/1000_round"], how="left")

    col_names = [
        "UncalAccelXMps2_avg",
        "UncalAccelYMps2_avg",
        "UncalAccelZMps2_avg",
        "UncalAccelXMps2_std",
        "UncalAccelYMps2_std",
        "UncalAccelZMps2_std",
        "UncalGyroXRadPerSec_avg",
        "UncalGyroYRadPerSec_avg",
        "UncalGyroZRadPerSec_avg",
        "UncalGyroXRadPerSec_std",
        "UncalGyroYRadPerSec_std",
        "UncalGyroZRadPerSec_std",
        "UncalMagXMicroT_avg",
        "UncalMagYMicroT_avg",
        "UncalMagZMicroT_avg",
        "UncalMagXMicroT_std",
        "UncalMagYMicroT_std",
        "UncalMagZMicroT_std"
    ]
    groups = data.groupby(["collectionName", "phoneName"])
    data[[col_name + "+1" for col_name in col_names]] = groups[col_names].shift(-1)

    nan_idxs = data[data["UncalAccelXMps2_avg+1"].isnull()].index
    data.loc[nan_idxs, [col_name + "+1" for col_name in col_names]] = data.loc[nan_idxs, col_names].values

    data["ratio"] = (data["utcTimeMillis"] % 1000) / 1000
    data[col_names] = data[col_names].values * (1 - data["ratio"]).values.reshape(-1, 1) + \
        data[[col_name + "+1" for col_name in col_names]].values * data["ratio"].values.reshape(-1, 1)

    return data

test_data = sensor_data_processing(test_gyro_data, test_mag_data, test_accel_data, test_data)
del test_gyro_data
del test_mag_data
del test_accel_data
gc.collect()


# 卡尔曼相关特征
kf = KF()
kf.apply_kf_smoothing(test_data, suffix="_kf")

def add_kf_features(data):
    data["dist_between_base_and_kf"] = calc_haversine(data.latDeg, data.lngDeg, data.latDeg_kf, data.lngDeg_kf)
    data["base_kf_lat_delta"] = data["latDeg"] - data["latDeg_kf"]
    data["base_kf_lng_delta"] = data["lngDeg"] - data["lngDeg_kf"]
    data["abs_base_kf_lng_delta"] = abs(data["lngDeg"] - data["lngDeg_kf"])
    data["abs_base_kf_lat_delta"] = abs(data["latDeg"] - data["latDeg_kf"])

add_kf_features(test_data)

# 缺失值均值填充
col_names_ = [
    "UncalAccelXMps2_avg",
    "UncalAccelYMps2_avg",
    "UncalAccelZMps2_avg",
    "UncalAccelXMps2_std",
    "UncalAccelYMps2_std",
    "UncalAccelZMps2_std",
    "UncalGyroXRadPerSec_avg",
    "UncalGyroYRadPerSec_avg",
    "UncalGyroZRadPerSec_avg",
    "UncalGyroXRadPerSec_std",
    "UncalGyroYRadPerSec_std",
    "UncalGyroZRadPerSec_std",
    "UncalMagXMicroT_avg",
    "UncalMagYMicroT_avg",
    "UncalMagZMicroT_avg",
    "UncalMagXMicroT_std",
    "UncalMagYMicroT_std",
    "UncalMagZMicroT_std"
]

test_data[col_names_] = test_data[col_names_].fillna(test_data[col_names_].mean())



test gnsslog parsing ...
../input/google-smartphone-decimeter-challenge/test/2021-04-29-US-MTV-2/SamsungS20Ultra/SamsungS20Ultra_GnssLog.txt


100%|██████████| 812203/812203 [00:01<00:00, 505961.07it/s]


../input/google-smartphone-decimeter-challenge/test/2021-04-29-US-MTV-2/Pixel5/Pixel5_GnssLog.txt


100%|██████████| 691517/691517 [00:01<00:00, 487102.77it/s]


../input/google-smartphone-decimeter-challenge/test/2021-04-29-US-MTV-2/Pixel4/Pixel4_GnssLog.txt


...

48it [00:19,  2.50it/s]
0it [00:00, ?it/s]

Reading UncalMag.csv from test ...


48it [00:10,  4.63it/s]
0it [00:00, ?it/s]

Reading UncalAccel.csv from test ...


48it [00:19,  2.52it/s]
100%|██████████| 48/48 [00:45<00:00,  1.06it/s]


In [7]:
check_random()

In [8]:
# test_data

In [9]:
cat_cols = [
    'correctedPrm_avg_bucketized',
    'rawPrUncM_avg_bucketized',
    'satClkDriftMps_avg_bucketized',
    'correctedPrm_std_bucketized',
    'rawPrUncM_std_bucketized',
    'satClkDriftMps_std_bucketized',
    "phoneCat"
]

numerical_cols = [
     'latDeg',
     'lngDeg',
     'heightAboveWgs84EllipsoidM',
     'ratio_of_year',
     'mean_latDeg+-1',
     'mean_lngDeg+-1',
     'latDeg_mean_delta_1',
     'lngDeg_mean_delta_1',
     'mean_latDeg+-2',
     'mean_lngDeg+-2',
     'latDeg_mean_delta_2',
     'lngDeg_mean_delta_2',
     'mean_latDeg+-3',
     'mean_lngDeg+-3',
     'latDeg_mean_delta_3',
     'lngDeg_mean_delta_3',
     'mean_latDeg+-4',
     'mean_lngDeg+-4',
     'latDeg_mean_delta_4',
     'lngDeg_mean_delta_4',
     'mean_latDeg+-5',
     'mean_lngDeg+-5',
     'latDeg_mean_delta_5',
     'lngDeg_mean_delta_5',
     'latDeg_pre_increment',
     'lngDeg_pre_increment',
     'latDeg_post_increment',
     'lngDeg_post_increment',
     'dist_pre',
     'dist_post',
     'latDeg_pre_post_mean_abs_delta',
     'lngDeg_pre_post_mean_abs_delta',
     'pre_post_mean_abs_dist',
     'max_latDeg_with_same_millisSinceGpsEpoch',
     'max_lngDeg_with_same_millisSinceGpsEpoch',
     'max_heightAboveWgs84EllipsoidM_with_same_millisSinceGpsEpoch',
     'min_latDeg_with_same_millisSinceGpsEpoch',
     'min_lngDeg_with_same_millisSinceGpsEpoch',
     'min_heightAboveWgs84EllipsoidM_with_same_millisSinceGpsEpoch',
     'mean_latDeg_with_same_millisSinceGpsEpoch',
     'mean_lngDeg_with_same_millisSinceGpsEpoch',
     'mean_heightAboveWgs84EllipsoidM_with_same_millisSinceGpsEpoch',
     'sum_latDeg_with_same_millisSinceGpsEpoch',
     'sum_lngDeg_with_same_millisSinceGpsEpoch',
     'sum_heightAboveWgs84EllipsoidM_with_same_millisSinceGpsEpoch',
     'count_latDeg_with_same_millisSinceGpsEpoch',
     'dist_between_baseline_and_phone_mean',
     'Cn0DbHz_avg',
     'BiasUncertaintyNanos_avg',
     'Cn0DbHz_std',
     'BiasUncertaintyNanos_std',
     'UncalAccelXMps2_avg',
     'UncalAccelYMps2_avg',
     'UncalAccelZMps2_avg',
     'UncalAccelXMps2_std',
     'UncalAccelYMps2_std',
     'UncalAccelZMps2_std',
     'UncalGyroXRadPerSec_avg',
     'UncalGyroYRadPerSec_avg',
     'UncalGyroZRadPerSec_avg',
     'UncalGyroXRadPerSec_std',
     'UncalGyroYRadPerSec_std',
     'UncalGyroZRadPerSec_std',
     'UncalMagXMicroT_avg',
     'UncalMagYMicroT_avg',
     'UncalMagZMicroT_avg',
     'UncalMagXMicroT_std',
     'UncalMagYMicroT_std',
     'UncalMagZMicroT_std',
     'latDeg_kf',
     'lngDeg_kf',
     'dist_between_base_and_kf',
     'base_kf_lat_delta',
     'base_kf_lng_delta',
     'abs_base_kf_lng_delta',
     'abs_base_kf_lat_delta'
]

feature_cols = [
     'latDeg',
     'lngDeg',
     'heightAboveWgs84EllipsoidM',
     'phoneCat',
     'ratio_of_year',
     'mean_latDeg+-1',
     'mean_lngDeg+-1',
     'latDeg_mean_delta_1',
     'lngDeg_mean_delta_1',
     'mean_latDeg+-2',
     'mean_lngDeg+-2',
     'latDeg_mean_delta_2',
     'lngDeg_mean_delta_2',
     'mean_latDeg+-3',
     'mean_lngDeg+-3',
     'latDeg_mean_delta_3',
     'lngDeg_mean_delta_3',
     'mean_latDeg+-4',
     'mean_lngDeg+-4',
     'latDeg_mean_delta_4',
     'lngDeg_mean_delta_4',
     'mean_latDeg+-5',
     'mean_lngDeg+-5',
     'latDeg_mean_delta_5',
     'lngDeg_mean_delta_5',
     'latDeg_pre_increment',
     'lngDeg_pre_increment',
     'latDeg_post_increment',
     'lngDeg_post_increment',
     'dist_pre',
     'dist_post',
     'latDeg_pre_post_mean_abs_delta',
     'lngDeg_pre_post_mean_abs_delta',
     'pre_post_mean_abs_dist',
     'max_latDeg_with_same_millisSinceGpsEpoch',
     'max_lngDeg_with_same_millisSinceGpsEpoch',
     'max_heightAboveWgs84EllipsoidM_with_same_millisSinceGpsEpoch',
     'min_latDeg_with_same_millisSinceGpsEpoch',
     'min_lngDeg_with_same_millisSinceGpsEpoch',
     'min_heightAboveWgs84EllipsoidM_with_same_millisSinceGpsEpoch',
     'mean_latDeg_with_same_millisSinceGpsEpoch',
     'mean_lngDeg_with_same_millisSinceGpsEpoch',
     'mean_heightAboveWgs84EllipsoidM_with_same_millisSinceGpsEpoch',
     'sum_latDeg_with_same_millisSinceGpsEpoch',
     'sum_lngDeg_with_same_millisSinceGpsEpoch',
     'sum_heightAboveWgs84EllipsoidM_with_same_millisSinceGpsEpoch',
     'count_latDeg_with_same_millisSinceGpsEpoch',
     'dist_between_baseline_and_phone_mean',
     'correctedPrm_avg_bucketized',
     'rawPrUncM_avg_bucketized',
     'satClkDriftMps_avg_bucketized',
     'correctedPrm_std_bucketized',
     'rawPrUncM_std_bucketized',
     'satClkDriftMps_std_bucketized',
     'Cn0DbHz_avg',
     'BiasUncertaintyNanos_avg',
     'Cn0DbHz_std',
     'BiasUncertaintyNanos_std',
     'UncalAccelXMps2_avg',
     'UncalAccelYMps2_avg',
     'UncalAccelZMps2_avg',
     'UncalAccelXMps2_std',
     'UncalAccelYMps2_std',
     'UncalAccelZMps2_std',
     'UncalGyroXRadPerSec_avg',
     'UncalGyroYRadPerSec_avg',
     'UncalGyroZRadPerSec_avg',
     'UncalGyroXRadPerSec_std',
     'UncalGyroYRadPerSec_std',
     'UncalGyroZRadPerSec_std',
     'UncalMagXMicroT_avg',
     'UncalMagYMicroT_avg',
     'UncalMagZMicroT_avg',
     'UncalMagXMicroT_std',
     'UncalMagYMicroT_std',
     'UncalMagZMicroT_std',
     'latDeg_kf',
     'lngDeg_kf',
     'dist_between_base_and_kf',
     'base_kf_lat_delta',
     'base_kf_lng_delta',
     'abs_base_kf_lng_delta',
     'abs_base_kf_lat_delta'
]

In [10]:
test = test_data[feature_cols]

test_num = test[numerical_cols]

test_cat_one_hot_1 = pd.get_dummies(test[[cat_cols[0]]].astype('str'))
test_cat_one_hot_2 = pd.get_dummies(test[[cat_cols[1]]].astype('str'))
test_cat_one_hot_3 = pd.get_dummies(test[[cat_cols[2]]].astype('str'))
test_cat_one_hot_4 = pd.get_dummies(test[[cat_cols[3]]].astype('str'))
test_cat_one_hot_5 = pd.get_dummies(test[[cat_cols[4]]].astype('str'))
test_cat_one_hot_6 = pd.get_dummies(test[[cat_cols[5]]].astype('str'))
test_cat_one_hot_7 = pd.get_dummies(test[[cat_cols[6]]].astype('str'))

In [11]:
from sklearn import preprocessing
# ss = preprocessing.StandardScaler()
ss = joblib.load('../input/ssssssss/ss.pkl')
test_num = ss.transform(test_num)

In [12]:
test_num

array([[-0.18031011,  0.4050528 , -0.6437015 , ..., -0.44753414,
         0.3872363 ,  0.04404465],
       [-0.1800925 ,  0.40514187, -0.65774996, ..., -0.14794978,
         0.08708674,  0.08119901],
       [-0.18001512,  0.40515064, -0.65630573, ...,  0.00692914,
        -0.05482815,  0.09850579],
       ...,
       [-1.17258624,  1.63775955, -0.29432811, ...,  0.00786696,
        -0.05388855, -0.03843269],
       [-1.17272043,  1.63775887, -0.31559774, ..., -0.02954099,
        -0.03154544, -0.00825561],
       [-1.1724593 ,  1.63787494, -0.25257661, ...,  0.00688437,
        -0.054873  , -0.03731724]])

In [13]:
epochs = 200
batch_size = 1024
learning_rate = 0.005

In [14]:
def r2(y_true, y_pred):
    a = K.square(y_pred - y_true)
    b = K.sum(a)
    c = K.mean(y_true)
    d = K.square(y_true - c)
    e = K.sum(d)
    f = 1 - b / e
    return f 

def my_loss(y_true, y_pred):
    loss_less = 1.0 
    loss_more = 2.0 
    
    error = y_pred - y_true
    error_fix = tf.where(tf.greater(y_pred, y_true), error * loss_less, - error * loss_more)
    mae_fix = K.mean(error_fix)
    return mae_fix

def check_random(SEED=2021):
    os.environ['TF_DETERMINISTIC_OPS'] = '1'
    os.environ['PYTHONHASHSEED'] = str(SEED)
    random.seed(SEED)
    np.random.seed(SEED)
    tf.random.set_seed(SEED)

def nns(x, hidden_units, activation, dropout_rates):
    for i in range(len(hidden_units)):
        x = Dense(hidden_units[i])(x)
        x = BatchNormalization()(x)
        x = Activation(activation[i])(x)
        x = Dropout(dropout_rates[i + 1])(x)
    return x     

def create_nn_model_new(num_cols, cat_cols_list, output_dim, learning_rate):
    inp = Input(shape=(num_cols,))

    inp_cat1 = Input(shape=(cat_cols_list[0],))
    inp_cat2 = Input(shape=(cat_cols_list[1],))
    inp_cat3 = Input(shape=(cat_cols_list[2],))
    inp_cat4 = Input(shape=(cat_cols_list[3],))
    inp_cat5 = Input(shape=(cat_cols_list[4],))
    inp_cat6 = Input(shape=(cat_cols_list[5],))
    inp_cat7 = Input(shape=(cat_cols_list[6],))

    dense_cats = []
    dense_cats.append(Dense(50)(inp_cat1))
    dense_cats.append(Dense(50)(inp_cat2))
    dense_cats.append(Dense(50)(inp_cat3))
    dense_cats.append(Dense(50)(inp_cat4))
    dense_cats.append(Dense(50)(inp_cat5))
    dense_cats.append(Dense(50)(inp_cat6))
    dense_cats.append(Dense(50)(inp_cat7))
    
    x = Concatenate(axis=-1)(dense_cats + [inp])
    x = Dense(512)(x)
    x = Activation(relu)(x)
    
    x = Dense(512)(x)
    x = Activation(relu)(x)
    
    x = Dense(256)(x)
    x = Activation(relu)(x)
    
    x = Dense(128)(x)
    x = Activation(relu)(x)
    
    x = Dense(64)(x)

    out = Dense(output_dim)(x)

    model = Model(inputs=[inp, 
                        inp_cat1, inp_cat2, 
                        inp_cat3, inp_cat4,
                        inp_cat5, inp_cat6, 
                        inp_cat7], outputs=out)
    model.compile(
        optimizer=AdamW(learning_rate=learning_rate, weight_decay=0.0001),
        loss=[my_loss],
        metrics=['mae', 'mse', r2]
    )

    return model

In [15]:
ann_dis = create_nn_model_new(test_num.shape[-1], [21, 21, 21, 21, 21, 21, 7], 1, learning_rate)
ann_dis.summary()
ann_dis.load_weights('../input/test-model/dis_ann.hdf5')

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            [(None, 21)]         0                                            
__________________________________________________________________________________________________
input_3 (InputLayer)            [(None, 21)]         0                                            
__________________________________________________________________________________________________
input_4 (InputLayer)            [(None, 21)]         0                                            
__________________________________________________________________________________________________
input_5 (InputLayer)            [(None, 21)]         0                                            
______________________________________________________________________________________________

In [16]:
ann_speed = create_nn_model_new(test_num.shape[-1], [21, 21, 21, 21, 21, 21, 7], 1, learning_rate)
ann_speed.summary()
ann_speed.load_weights('../input/test-model/speed_ann.hdf5')

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_10 (InputLayer)           [(None, 21)]         0                                            
__________________________________________________________________________________________________
input_11 (InputLayer)           [(None, 21)]         0                                            
__________________________________________________________________________________________________
input_12 (InputLayer)           [(None, 21)]         0                                            
__________________________________________________________________________________________________
input_13 (InputLayer)           [(None, 21)]         0                                            
____________________________________________________________________________________________

In [17]:
pred_dis = ann_dis.predict([test_num, 
                            test_cat_one_hot_1, test_cat_one_hot_2, test_cat_one_hot_3, 
                            test_cat_one_hot_4, test_cat_one_hot_5, test_cat_one_hot_6 ,test_cat_one_hot_7])
pred_speed = ann_speed.predict([test_num, 
                            test_cat_one_hot_1, test_cat_one_hot_2, test_cat_one_hot_3, 
                            test_cat_one_hot_4, test_cat_one_hot_5, test_cat_one_hot_6 ,test_cat_one_hot_7])

In [18]:
test['pred_dist_between_baseline_and_truth'] = pred_dis
test['pred_speed_between_baseline_and_truth'] = pred_speed

In [19]:
test[['pred_dist_between_baseline_and_truth', 'pred_speed_between_baseline_and_truth']].describe()

,pred_dist_between_baseline_and_truth,pred_speed_between_baseline_and_truth
count,91486.000000,91486.000000
mean,3.768844,2.599115
std,4.195992,4.431914
min,0.876034,0.145333
25%,1.729072,0.896066
50%,2.535623,1.484980
75%,4.449728,2.805116
max,397.660675,357.005249


# 分三次融合处理

## 1.lgb 7.125 14.25 16 13

In [20]:
test_data_lgb = test_data.copy()

In [21]:
th_speed_normal = (6.75 + 7.5) / 2 
th_dist_normal = (13.5 + 15) / 2
# th_speed_normal = 6
# th_dist_normal = 12

In [22]:
regression = test_data_lgb.loc[((test['pred_dist_between_baseline_and_truth'] <= th_dist_normal) & (test['pred_speed_between_baseline_and_truth'] <= th_speed_normal))].copy()

In [23]:
model_lat = joblib.load('../input/lgb-regression-after-keras-ann-dist-speed/model_lat.pkl')
model_lng = joblib.load('../input/lgb-regression-after-keras-ann-dist-speed/model_lng.pkl')

In [24]:
dis_lat = model_lat.predict(regression[feature_cols])
dis_lng = model_lng.predict(regression[feature_cols])

In [25]:
test_data_lgb.loc[((test['pred_dist_between_baseline_and_truth'] <= th_dist_normal) & (test['pred_speed_between_baseline_and_truth'] <= th_speed_normal)), 'latDeg'] -= dis_lat
test_data_lgb.loc[((test['pred_dist_between_baseline_and_truth'] <= th_dist_normal) & (test['pred_speed_between_baseline_and_truth'] <= th_speed_normal)), 'lngDeg'] -= dis_lng

In [26]:
# tmp = test_data_lgb.copy()
# th_speed_abnormal = 16
# th_dist_abnormal = 13
# print(len(tmp.loc[((test['pred_dist_between_baseline_and_truth'] > th_dist_abnormal) | (test['pred_speed_between_baseline_and_truth'] > th_speed_abnormal)), :]))


In [27]:
kf = KF()

tmp = test_data_lgb.copy()

th_speed_abnormal = 16
th_dist_abnormal = 13

# th_speed_abnormal = th_speed_normal
# th_dist_abnormal = th_dist_normal

print(len(tmp.loc[((test['pred_dist_between_baseline_and_truth'] > th_dist_abnormal) | (test['pred_speed_between_baseline_and_truth'] > th_speed_abnormal)), :]))
tmp.loc[((test['pred_dist_between_baseline_and_truth'] > th_dist_abnormal) | (test['pred_speed_between_baseline_and_truth'] > th_speed_abnormal)), ['latDeg', 'lngDeg']] = np.nan

2606


In [28]:
kf.apply_kf_smoothing(tmp, "")
smoothed_tmp = mean_with_other_phones(tmp)

100%|██████████| 48/48 [00:42<00:00,  1.12it/s]


In [29]:
data_sub = pd.read_csv('../input/google-smartphone-decimeter-challenge' + '/' + 'sample_submission.csv')
df_sub_lgb_best = data_sub[['phone', 'millisSinceGpsEpoch']].merge(smoothed_tmp[['phone', 'millisSinceGpsEpoch', 'latDeg', 'lngDeg']], on=['phone', 'millisSinceGpsEpoch'], how='inner')

In [30]:
# df_sub.to_csv("./submission1.csv", index=False)

In [31]:
df_sub_lgb_best

,phone,millisSinceGpsEpoch,latDeg,lngDeg
0,2020-05-15-US-MTV-1_Pixel4,1273608785432,37.416577,-122.082019
1,2020-05-15-US-MTV-1_Pixel4,1273608786432,37.416588,-122.082043
2,2020-05-15-US-MTV-1_Pixel4,1273608787432,37.416597,-122.082058
3,2020-05-15-US-MTV-1_Pixel4,1273608788432,37.416602,-122.082066
4,2020-05-15-US-MTV-1_Pixel4,1273608789432,37.416606,-122.082069
...,...,...,...,...
91481,2021-04-29-US-SJC-3_SamsungS20Ultra,1303763185000,37.334561,-121.899392
91482,2021-04-29-US-SJC-3_SamsungS20Ultra,1303763186000,37.334562,-121.899384
91483,2021-04-29-US-SJC-3_SamsungS20Ultra,1303763187000,37.334564,-121.899375
91484,2021-04-29-US-SJC-3_SamsungS20Ultra,1303763188000,37.334566,-121.899367


## lgb 16 12 16 13

In [32]:
test_data_forest = test_data.copy()

In [33]:
th_speed_normal = 16
th_dist_normal = 12

In [34]:
regression = test_data_forest.loc[((test['pred_dist_between_baseline_and_truth'] <= th_dist_normal) & (test['pred_speed_between_baseline_and_truth'] <= th_speed_normal))].copy()

In [35]:
model_lat_forest = joblib.load('../input/fork-of-lgb-regression-16-12-16-13/model_lat_16_12_16_13.pkl')
model_lng_forest = joblib.load('../input/fork-of-lgb-regression-16-12-16-13/model_lng_16_12_16_13.pkl')

In [36]:
dis_lat = model_lat_forest.predict(regression[feature_cols])
dis_lng = model_lng_forest.predict(regression[feature_cols])

In [37]:
test_data_forest.loc[((test['pred_dist_between_baseline_and_truth'] <= th_dist_normal) & (test['pred_speed_between_baseline_and_truth'] <= th_speed_normal)), 'latDeg'] -= dis_lat
test_data_forest.loc[((test['pred_dist_between_baseline_and_truth'] <= th_dist_normal) & (test['pred_speed_between_baseline_and_truth'] <= th_speed_normal)), 'lngDeg'] -= dis_lng

In [38]:
kf = KF()

tmp = test_data_forest.copy()

th_speed_abnormal = 16
th_dist_abnormal = 13

# th_speed_abnormal = th_speed_normal
# th_dist_abnormal = th_dist_normal

print(len(tmp.loc[((test['pred_dist_between_baseline_and_truth'] > th_dist_abnormal) | (test['pred_speed_between_baseline_and_truth'] > th_speed_abnormal)), :]))
tmp.loc[((test['pred_dist_between_baseline_and_truth'] > th_dist_abnormal) | (test['pred_speed_between_baseline_and_truth'] > th_speed_abnormal)), ['latDeg', 'lngDeg']] = np.nan

2606


In [39]:
kf.apply_kf_smoothing(tmp, "")
smoothed_tmp = mean_with_other_phones(tmp)

100%|██████████| 48/48 [00:43<00:00,  1.10it/s]


In [40]:
data_sub1 = data_sub.copy()
df_sub_forest = data_sub1[['phone', 'millisSinceGpsEpoch']].merge(smoothed_tmp[['phone', 'millisSinceGpsEpoch', 'latDeg', 'lngDeg']], on=['phone', 'millisSinceGpsEpoch'], how='inner')

In [41]:
df_sub_forest

,phone,millisSinceGpsEpoch,latDeg,lngDeg
0,2020-05-15-US-MTV-1_Pixel4,1273608785432,37.416579,-122.082018
1,2020-05-15-US-MTV-1_Pixel4,1273608786432,37.416589,-122.082042
2,2020-05-15-US-MTV-1_Pixel4,1273608787432,37.416596,-122.082058
3,2020-05-15-US-MTV-1_Pixel4,1273608788432,37.416600,-122.082067
4,2020-05-15-US-MTV-1_Pixel4,1273608789432,37.416603,-122.082072
...,...,...,...,...
91481,2021-04-29-US-SJC-3_SamsungS20Ultra,1303763185000,37.334560,-121.899391
91482,2021-04-29-US-SJC-3_SamsungS20Ultra,1303763186000,37.334561,-121.899382
91483,2021-04-29-US-SJC-3_SamsungS20Ultra,1303763187000,37.334564,-121.899374
91484,2021-04-29-US-SJC-3_SamsungS20Ultra,1303763188000,37.334566,-121.899366


## lgb 16 13 16 13

In [42]:
test_data_lgbworse = test_data.copy()

In [43]:
th_speed_normal = 16
th_dist_normal = 13

In [44]:
regression = test_data_lgbworse.loc[((test['pred_dist_between_baseline_and_truth'] <= th_dist_normal) & (test['pred_speed_between_baseline_and_truth'] <= th_speed_normal))].copy()

In [45]:
model_lat_lgbworse = joblib.load('../input/lgb-regression-16-13-16-13/model_lat_16_13_16_13.pkl')
model_lng_lgbworse = joblib.load('../input/lgb-regression-16-13-16-13/model_lng_16_13_16_13.pkl')

In [46]:
dis_lat = model_lat_lgbworse.predict(regression[feature_cols])
dis_lng = model_lng_lgbworse.predict(regression[feature_cols])

In [47]:
test_data_lgbworse.loc[((test['pred_dist_between_baseline_and_truth'] <= th_dist_normal) & (test['pred_speed_between_baseline_and_truth'] <= th_speed_normal)), 'latDeg'] -= dis_lat
test_data_lgbworse.loc[((test['pred_dist_between_baseline_and_truth'] <= th_dist_normal) & (test['pred_speed_between_baseline_and_truth'] <= th_speed_normal)), 'lngDeg'] -= dis_lng

In [48]:
kf = KF()

tmp = test_data_lgbworse.copy()

th_speed_abnormal = 16
th_dist_abnormal = 13

# th_speed_abnormal = th_speed_normal
# th_dist_abnormal = th_dist_normal

print(len(tmp.loc[((test['pred_dist_between_baseline_and_truth'] > th_dist_abnormal) | (test['pred_speed_between_baseline_and_truth'] > th_speed_abnormal)), :]))
tmp.loc[((test['pred_dist_between_baseline_and_truth'] > th_dist_abnormal) | (test['pred_speed_between_baseline_and_truth'] > th_speed_abnormal)), ['latDeg', 'lngDeg']] = np.nan

2606


In [49]:
kf.apply_kf_smoothing(tmp, "")
smoothed_tmp = mean_with_other_phones(tmp)

100%|██████████| 48/48 [00:46<00:00,  1.03it/s]


In [50]:
data_sub2 = data_sub.copy()
df_sub_lgbworse = data_sub2[['phone', 'millisSinceGpsEpoch']].merge(smoothed_tmp[['phone', 'millisSinceGpsEpoch', 'latDeg', 'lngDeg']], on=['phone', 'millisSinceGpsEpoch'], how='inner')

In [51]:
df_sub_lgbworse

,phone,millisSinceGpsEpoch,latDeg,lngDeg
0,2020-05-15-US-MTV-1_Pixel4,1273608785432,37.416577,-122.082018
1,2020-05-15-US-MTV-1_Pixel4,1273608786432,37.416588,-122.082043
2,2020-05-15-US-MTV-1_Pixel4,1273608787432,37.416596,-122.082058
3,2020-05-15-US-MTV-1_Pixel4,1273608788432,37.416600,-122.082067
4,2020-05-15-US-MTV-1_Pixel4,1273608789432,37.416603,-122.082071
...,...,...,...,...
91481,2021-04-29-US-SJC-3_SamsungS20Ultra,1303763185000,37.334560,-121.899390
91482,2021-04-29-US-SJC-3_SamsungS20Ultra,1303763186000,37.334562,-121.899381
91483,2021-04-29-US-SJC-3_SamsungS20Ultra,1303763187000,37.334564,-121.899373
91484,2021-04-29-US-SJC-3_SamsungS20Ultra,1303763188000,37.334567,-121.899365


In [52]:
# df_sub2 = df_sub.copy()
# df_sub2['latDeg'] = (df_sub['latDeg'] **2 * 0.5 + df_sub1['latDeg'] **2 * 0.5)**0.5
# df_sub2['lngDeg'] = -(df_sub['lngDeg'] **2 * 0.5 + df_sub1['lngDeg'] **2 * 0.5)**0.5

# blend 融合

In [53]:
# gsdc_position_shift = pd.read_csv('./结果保存/5.002.csv')
# gsdc_phones_mean_prediction = pd.read_csv('./结果保存/4.936.csv')
# google_smartphone_decimeter_challenge = pd.read_csv('./结果保存/4.970.csv')
gsdc_position_shift = df_sub_forest
gsdc_phones_mean_prediction = df_sub_lgb_best
google_smartphone_decimeter_challenge = df_sub_lgbworse

In [54]:
blend_weights_lat = {
    "gsdc-phones-mean-prediction": (7.5 + 7.625) / 2 + 0.0625 + 0.0625/2 - 0.0625/4 + 2*0.0625/8,
    "device-eda-interpolate-by-removing-device-en-ja":  (2.25 + 2.5) / 2 + (2.25 + 2.5) / 4,
    "gsdc-position-shift":  0.75 - 0.125,
}


blend_weights_long = {
    "gsdc-phones-mean-prediction": (7.5 + 7.625) / 2 + 0.0625  + 0.0625/2 - 0.0625/4 + 0.0625/8,
    "device-eda-interpolate-by-removing-device-en-ja":(2.25 + 2.5) / 2 + (2.25 + 2.5) / 4,
    "gsdc-position-shift": 0.75 - 0.125,
}


lat_norm = sum(blend_weights_lat.values())
long_norm = sum(blend_weights_long.values())

blend = gsdc_phones_mean_prediction
# blend['millisSinceGpsEpoch'] = (
#     gsdc_position_shift['millisSinceGpsEpoch'] * blend_weights["gsdc-position-shift"]/norm +
#     gsdc_phones_mean_prediction['millisSinceGpsEpoch'] * blend_weights["gsdc-phones-mean-prediction"]/norm +
#     google_smartphone_decimeter_challenge['millisSinceGpsEpoch'] * blend_weights["device-eda-interpolate-by-removing-device-en-ja"]/norm
# )


blend['latDeg'] = (
    gsdc_position_shift['latDeg'] **2 * blend_weights_lat["gsdc-position-shift"]/lat_norm +
    gsdc_phones_mean_prediction['latDeg'] **2 * blend_weights_lat["gsdc-phones-mean-prediction"]/lat_norm +
    google_smartphone_decimeter_challenge['latDeg']**2  * blend_weights_lat["device-eda-interpolate-by-removing-device-en-ja"]/lat_norm 
)**0.5

blend['lngDeg'] = (
    gsdc_position_shift['lngDeg'] * blend_weights_long["gsdc-position-shift"]/long_norm +
    gsdc_phones_mean_prediction['lngDeg'] * blend_weights_long["gsdc-phones-mean-prediction"]/long_norm +
    google_smartphone_decimeter_challenge['lngDeg'] * blend_weights_long["device-eda-interpolate-by-removing-device-en-ja"]/long_norm 
)

## 最终输出

In [55]:
blend.to_csv("submission.csv", index=False)

In [56]:
blend

,phone,millisSinceGpsEpoch,latDeg,lngDeg
0,2020-05-15-US-MTV-1_Pixel4,1273608785432,37.416577,-122.082019
1,2020-05-15-US-MTV-1_Pixel4,1273608786432,37.416588,-122.082043
2,2020-05-15-US-MTV-1_Pixel4,1273608787432,37.416597,-122.082058
3,2020-05-15-US-MTV-1_Pixel4,1273608788432,37.416601,-122.082066
4,2020-05-15-US-MTV-1_Pixel4,1273608789432,37.416605,-122.082070
...,...,...,...,...
91481,2021-04-29-US-SJC-3_SamsungS20Ultra,1303763185000,37.334561,-121.899391
91482,2021-04-29-US-SJC-3_SamsungS20Ultra,1303763186000,37.334562,-121.899383
91483,2021-04-29-US-SJC-3_SamsungS20Ultra,1303763187000,37.334564,-121.899374
91484,2021-04-29-US-SJC-3_SamsungS20Ultra,1303763188000,37.334566,-121.899366


In [57]:
!rm -rf test